# LLM tooling for Chemistry

## *Gemini*

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
September 2025

-----

## Introduction

This notebook shows how to do parallel [Function Calling](https://ai.google.dev/gemini-api/docs/function-calling) workflows with Large Language Models (LLMs) of Gemini. 

The Raku package ["WWW::Gemini"](https://github.com/antononcube/Raku-WWW-Gemini), [AAp2], is used.

### Examples and big picture

The rest of the notebook gives concrete code how to do streamline multiple-tool function calling with Gemini's LLMs using Raku. 
Gemini's function calling example ["Parallel Function Calling"](https://ai.google.dev/gemini-api/docs/function-calling#parallel_function_calling), [Gem1], is followed.

This notebook belongs to a collection of notebooks describing how to do LLM function calling with Raku.

Compared to the previously described LLM workflows with OpenAI, [[AA1](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-1-openai/)], and Gemini, [[AA2](https://rakuforprediction.wordpress.com/2025/06/07/llm-function-calling-workflows-part-2-googles-gemini/)], the Gemini LLM workflow in this notebook demonstrates:

- Use of multiple tools (parallel function calling)
- Automatic generation of hashmap (or JSON) tool descriptors
- Streamlined computation of multiple tool results from multiple LLM requests

The streamlining is achieved by using the provided by ["LLM::Functions"](https://raku.land/zef:antononcube/LLM::Functions), [AAp3]:

- Classes `LLM::Tool`, `LLM::ToolRequest`, and `LLM::ToolResult`
- Subs `llm-tool-definition` and `generate-llm-tool-result`
    - The former sub leverages Raku's introspection features. 
    - The latter sub matches tools and requests in order to compute tool responses.

-----

## Setup

Load packages:

In [1]:
use JSON::Fast;
use Data::Reshapers;
use Data::TypeSystem;
use LLM::Tooling;
use LLM::Functions;
use LLM::Functions::TooledGemini;
use LLM::Graph;
use Chemistry::Stoichiometry;
use WWW::Gemini;

Choose a model:

In [2]:
my $model = "gemini-2.0-flash";

gemini-2.0-flash

----

## Diagram

Here is a [Mermaid-JS](https://mermaid.js.org) diagram that shows _single-pass_ LLM-and-tool interaction:


```mermaid
sequenceDiagram
    participant Developer
    participant Model as LLM Service (Gemini)
    participant Tool as External Function

    Note over Developer, Model: 1. Tool Definitions + Messages
    Developer->>Model: molecular-mass(molecule) <br> "What's the mass of SO2?"

    Note over Model: 2. Tool Calls
    Model->>Tool: molecular-mass("SO2")

    Note over Tool: 3. Execute Function Code
    Tool->>Model: {"mass": 64.058}

    Note over Model: 4. Results
    Model->>Developer: {"mass": 64.058}

    Note over Developer: 5. Final Response
    Developer->>Model: The molecular mass of SO2 is 64.058.
```


**Remark:** Instead of one invocation of a local tool -- as shown in the diagram -- often enough multiple LLM-&-local-computer interactions are used for LLM-tool computation.


Here is an example of calling `molecular-mass`:

In [3]:
molecular-mass("SO2")

64.058

----

## Define package functions as tools

Define a few subs -- _tools_ -- with sub- and argument descriptions (i.e. attached Pod values, or [declarator blocks](https://docs.raku.org/language/pod#Declarator_blocks)):

In [ ]:
sink my $model = "gemini-2.0-flash";
sink my $format = 'json';

In [ ]:
sink my @tool-objects =
        LLM::Tool.new(&molecular-mass),
        LLM::Tool.new(&balance-chemical-equation)
        ;

In [ ]:
my @tool-specs =
{
    :name("molecular-mass"), 
    :description("Convert chemical compound formula (molecule) into molecule mass."), 
    :parameters(
        {
            :type("object")
            :properties( {"\$spec" => {:description("A molecule formula, or a list of molecule formulas"), :type("string")}}), 
            :required(["\$spec"]), 
        }), 
},
{
    :name("balance-chemical-equation"), 
    :description("Balance the given chemical equation."), 
    :parameters(
        {
            :type("object")
            :properties( {"\$spec" => {:description("A chemical equation to be balanced"), :type("string")}}), 
            :required(["\$spec"]), 
        }), 
},
;

deduce-type(@tool-specs)

Here are additional tool-mode configurations (see ["Function calling modes"](https://ai.google.dev/gemini-api/docs/function-calling?example=weather#function_calling_modes) of [Gem1]):

In [ ]:
my %tool-config =
  functionCallingConfig => {
    mode => "ANY",
    allowedFunctionNames => <molecular-mass balance-chemical-equation>
  };

-----

## LLM-tool invocation example **1**

A single prompt or an array of prompts:

In [ ]:
sink my $input = "What are the masses of SO2, O3, and C2H5OH? Also balance: C2H5OH + O2 = H2O + CO2";

LLM result:

In [34]:
LLM::Functions::TooledGemini.synthesize(
        $input,
        @tool-objects,
        :$model,
        :@tool-specs,
        format => 'text'):echo

LLM invocation : 1
response => {candidates => [{avgLogprobs => 5.257385529820686e-07, content => {parts => [{functionCall => {args => {$spec => SO2}, name => molecular-mass}} {functionCall => {args => {$spec => O3}, name => molecular-mass}} {functionCall => {args => {$spec => C2H5OH}, name => molecular-mass}} {functionCall => {args => {$spec => C2H5OH + O2 = H2O + CO2}, name => balance-chemical-equation}}], role => model}, finishReason => STOP, safetyRatings => [{category => HARM_CATEGORY_HATE_SPEECH, probability => NEGLIGIBLE} {category => HARM_CATEGORY_DANGEROUS_CONTENT, probability => NEGLIGIBLE} {category => HARM_CATEGORY_HARASSMENT, probability => NEGLIGIBLE} {category => HARM_CATEGORY_SEXUALLY_EXPLICIT, probability => NEGLIGIBLE}]}], modelVersion => gemini-2.0-flash, responseId => kPO9aNX2CZCtqtsPkLmbuQ4, usageMetadata => {candidatesTokenCount => 46, candidatesTokensDetails => [{modality => TEXT, tokenCount => 46}], promptTokenCount => 89, promptTokensDetails => [{modality => TEX

The masses of SO2, O3, and C2H5OH are 64.058, 47.997, and 46.069 respectively. The balanced chemical equation is 1 C2H5OH + 3 O2 -> 2 CO2 + 3 H2O.


--- 

## "Thoughtful" response

In [39]:
#% markdown
my $input = "How many molecules a kilogram of water has? Use LaTeX for the formulas. (If any.)";

LLM::Functions::TooledGemini.synthesize(
        $input,
        @tool-objects,
        :$model,
        :@tool-specs,
        format => 'text'):!echo

The molecular mass of water (\(H_2O\)) is approximately 18.015 atomic mass units (amu). To find out how many water molecules are in a kilogram of water, we need to use Avogadro's number (\(N_A\)), which is approximately \(6.022 \times 10^{23}\) molecules per mole.

First, convert 1 kilogram of water to grams:
1 kg = 1000 g

Next, calculate the number of moles of water in 1000 g:
\[
\text{moles of } H_2O = \frac{\text{mass of } H_2O}{\text{molecular mass of } H_2O} = \frac{1000 \text{ g}}{18.015 \text{ g/mol}} \approx 55.509 \text{ mol}
\]

Now, multiply the number of moles by Avogadro's number to find the number of molecules:
\[
\text{number of } H_2O \text{ molecules} = \text{moles of } H_2O \times N_A = 55.509 \text{ mol} \times 6.022 \times 10^{23} \text{ molecules/mol} \approx 3.343 \times 10^{25} \text{ molecules}
\]

So, there are approximately \(3.343 \times 10^{25}\) water molecules in a kilogram of water.


The molecular mass of water (\(H_2O\)) is approximately 18.015 atomic mass units (amu). To find out how many water molecules are in a kilogram of water, we need to use Avogadro's number (\(N_A\)), which is approximately \(6.022 \times 10^{23}\) molecules per mole.

First, convert 1 kilogram of water to grams:
1 kg = 1000 g

Next, calculate the number of moles of water in 1000 g:

$$
\text{moles of } H_2O = \frac{\text{mass of } H_2O}{\text{molecular mass of } H_2O} = \frac{1000 \text{ g}}{18.015 \text{ g/mol}} \approx 55.509 \text{ mol}
$$

Now, multiply the number of moles by Avogadro's number to find the number of molecules:
$$
\text{number of } H_2O \text{ molecules} = \text{moles of } H_2O \times N_A = 55.509 \text{ mol} \times 6.022 \times 10^{23} \text{ molecules/mol} \approx 3.343 \times 10^{25} \text{ molecules}
$$

So, there are approximately \(3.343 \times 10^{25}\) water molecules in a kilogram of water.

-----

## References

### Articles, blog posts

[AA1] Anton Antonov,
["LLM function calling workflows (Part 1, OpenAI)"](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-1-openai/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA2] Anton Antonov,
["LLM function calling workflows (Part 2, Google's Gemini)"](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-2-google-gemini/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA3] Anton Antonov,
["LLM function calling workflows (Part 3, Facilitation)"](https://rakuforprediction.wordpress.com/2025/06/08/llm-function-calling-workflows-part-3-facilitation/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[Gem1] Google Gemini,
["Gemini Developer API"](https://ai.google.dev/gemini-api/docs).

[WRI1] Wolfram Research, Inc.
["LLM-Related Functionality" guide](https://reference.wolfram.com/language/guide/LLMFunctions.html).



### Packages 

[AAp1] Anton Antonov,
[WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::Gemini Raku package](https://github.com/antononcube/Raku-WWW-Gemini),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).